# Make RNN learn to perform addition

改寫https://github.com/keras-team/keras/blob/master/examples/addition_rnn.py

手法為sequence to sequence learning

而目標為例如輸入535+61, 希望輸出為596

In [14]:
from keras.models import Model
from keras.layers import Dense, LSTM, Input, RepeatVector, TimeDistributed
from keras.activations import softmax
import numpy as np

In [15]:
class CharacterTable():
    def __init__(self, chars):
        """ 製造char->int以及int->char的對應
        @ Args:
            chars(str): 所有可能的輸入
        """
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))
    
    def encode(self, C, num_rows):
        """ 將給定的string C做one-hot encoding
        @ Args:
            C(str): 要做encoding的string
            num_rows(int): 即可能的輸入個數, 比如數字有10種可能, 此即為10
        @ returns:
            x(np.array): C的每個char經過encoding後的結果(num_rows x len(C))
        """
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1
        return x
    
    def decode(self, x, calc_argmax=True):
        """ 將int轉換回char
        """
        if calc_argmax:
            x = x.argmax(axis=-1)
        return ''.join(self.indices_char[x] for x in list(x))

In [16]:
class colors:
    ok = '\033[92m'
    fail = '\033[91m'
    close = '\033[0m'

In [17]:
NB_TRAINING_SAMPLES = 50000 # number of addition questions
NB_DIGITS = 3 # input number in [0 - 999]
REVERSE = True # like data augmentation, we have 535+61 and then we'll train 61+535 too
MAX_STRING_LENGTH = NB_DIGITS*2 + 1 # for '345+678', then length is 3*2+1=7

In [18]:
all_chars = '0123456789+ ' # 所有char的可能(12個)
char_table = CharacterTable(all_chars) 

In [19]:
questions = [] # 所有問題, 如'123+456'
answers = [] # 答案, 如579
seen = set() # 出現過的問題

In [20]:
MAXIMUM = 10**NB_DIGITS-1
while len(questions) < NB_TRAINING_SAMPLES:
    a = np.random.randint(0, MAXIMUM)
    b = np.random.randint(0, MAXIMUM)
    q1 = str(a) + '+' + str(b)
    # 這種放法nn可以簡單的知道"非空白之後高機率是非空白", 若調換則NN要多學判斷數字斷點的方式
    q1 = ' '*(MAX_STRING_LENGTH - len(q1)) + q1
    q2 = str(b) + '+' + str(a)
    q2 = ' '*(MAX_STRING_LENGTH - len(q2)) + q2
    if q1 in seen:
        continue
    seen.add(q1)
    ans = str(eval(q1))
    questions.append(q1)
    # 類似前面的方式, 這樣擺的話NN知道"空白之後高機率是空白"
    # 不過我沒驗證  或許實際上沒差?
    answers.append(ans + ' '*(NB_DIGITS+1-len(ans)))
    
    if REVERSE:
        seen.add(q2)    
        ans = str(eval(q2))        
        questions.append(q2)            
        answers.append(ans + ' '*(NB_DIGITS+1-len(ans)) )    

* The shape of x:
[問題數, 問題最大長度(即RNN的輸入長度), 所有的字符數(即one-hot向量的長度)]

In [21]:
x = np.zeros(shape=(len(questions), MAX_STRING_LENGTH, len(all_chars)), dtype=np.bool)
# NB_DIGITS+1: 三位數+三位數最多變四位數
y = np.zeros(shape=(len(questions), NB_DIGITS+1, len(all_chars)), dtype=np.bool)

for i, expression in enumerate(questions):
    x[i] = char_table.encode(expression, MAX_STRING_LENGTH)
for i, answer in enumerate(answers):
    y[i] = char_table.encode(answer, NB_DIGITS+1)

In [22]:
random_order = np.arange(len(questions))
np.random.shuffle(random_order)

x = x[random_order]
y = y[random_order]

In [23]:
# train-test split
split_index = int(0.9*len(questions))
train_x, val_x = x[:split_index], x[split_index:]
train_y, val_y = y[:split_index], y[split_index:]

In [24]:
HIDDEN_SIZE = 128 # lstm中hidden units的數目
NB_LSTM_LAYERS_WITH_SEQ_OUTPUT = 1 # encoder後串接的lstm層數
RNN_MODEL = LSTM

In [25]:
# input shape即為MAX_STRING_LENGTH(此例為3+3+1=7) x encoding長度(所有char的數目, 此例為12)
model_input = Input(shape=train_x.shape[1:], name='input_layer')
# 此處lstm遞迴MAX_STRING_LENGTH次, 並且只輸出這整個字串的encoder結果, 並不輸出sequence
rnn = RNN_MODEL(units=HIDDEN_SIZE)(model_input)
# 將上面結果重複NB_DIGITS+1次後再丟到LSTM中, 並且輸出sequence
# 即我們最終的目標是預測一個最長可能有NB_DIGITS+1的數字(e.g. 600+500=1100)
rnn = RepeatVector(NB_DIGITS+1)(rnn)
for _ in range(NB_LSTM_LAYERS_WITH_SEQ_OUTPUT):
    rnn = RNN_MODEL(HIDDEN_SIZE, return_sequences=True)(rnn)
# 現在依照初始的架構, 此處的shape為NB_DIGITS x HIDDEN_SIZE
# TimeDistributed對NB_DIGITS+1個資料分別建立一個NN
# 也就是TimeDistributed分配的網路數=RNN遞迴次數=輸出的sequence長度=NB_DIGITS+1
# 此目的在於將每個time step的資訊分開, 如最後面的圖的decoder部分中Dense彼此沒有連接
# 若沒有這個步驟, 則所有time step的資訊將會混在一起
# 由於每個char都有12種可能, 因此Dense的hidden unit數目為len(all_chars)=12
prediction = TimeDistributed(Dense(len(all_chars), activation='softmax'))(rnn)
model = Model(inputs=[model_input], outputs=[prediction])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_layer (InputLayer)     (None, 7, 12)             0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 128)               72192     
_________________________________________________________________
repeat_vector_2 (RepeatVecto (None, 4, 128)            0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 4, 128)            131584    
_________________________________________________________________
time_distributed_2 (TimeDist (None, 4, 12)             1548      
Total params: 205,324
Trainable params: 205,324
Non-trainable params: 0
_________________________________________________________________
None


In [26]:
BATCH_SIZE=32
EPOCHS=100
model.compile(optimizer='nadam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
for i in range(200):
    model.fit(x=train_x, y=train_y, validation_data=(val_x, val_y),
              batch_size=BATCH_SIZE, epochs=1)
    for i in range(10):
        ind = np.random.randint(0, len(val_x))
        rowx, rowy = val_x[np.array([ind])], val_y[np.array([ind])]
        preds = model.predict(rowx)
        q = char_table.decode(rowx[0])
        correct = char_table.decode(rowy[0])
        guess = char_table.decode(preds[0], calc_argmax=True)
        print('Q', q[::-1] if REVERSE else q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print(colors.ok + '☑' + colors.close, end=' ')
        else:
            print(colors.fail + '☒' + colors.close, end=' ')
        print(guess)

Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 21s 467us/step - loss: 1.6182 - acc: 0.4023 - val_loss: 1.4095 - val_acc: 0.4583
Q 575+356 T 1228 ☒ 1227
Q 337+237 T 1465 ☒ 1402
Q 679+984 T 1465 ☒ 1577
Q 366+63  T 699  ☒ 700 
Q 629+693 T 1322 ☒ 1300
Q 446+221 T 766  ☒ 707 
Q 268+136 T 1493 ☒ 1472
Q 388+92  T 912  ☒ 900 
Q 299+159 T 1943 ☒ 1777
Q 6+476   T 680  ☒ 707 
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 19s 426us/step - loss: 1.1247 - acc: 0.5770 - val_loss: 0.9865 - val_acc: 0.6322
Q 675+519 T 1491 ☒ 1499
Q 341+846 T 791  ☒ 794 
Q 132+91  T 250  ☒ 210 
Q 319+472 T 1187 ☒ 1177
Q 079+713 T 1287 ☒ 1299
Q 331+981 T 322  ☒ 214 
Q 291+516 T 807  ☒ 704 
Q 197+94  T 840  ☒ 844 
Q 035+122 T 751  ☒ 759 
Q 119+525 T 1436 ☒ 1446
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 19s 427us/step - loss: 0.7673 - acc: 0.7

45000/45000 [==============================] - 19s 428us/step - loss: 0.0204 - acc: 0.9939 - val_loss: 0.0143 - val_acc: 0.9954
Q 724+952 T 686  ☑ 686 
Q 515+571 T 690  ☑ 690 
Q 937+776 T 1416 ☑ 1416
Q 26+017  T 772  ☒ 872 
Q 736+729 T 1564 ☑ 1564
Q 902+374 T 682  ☑ 682 
Q 062+444 T 704  ☑ 704 
Q 356+081 T 833  ☑ 833 
Q 976+001 T 779  ☑ 779 
Q 713+635 T 853  ☑ 853 
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 19s 420us/step - loss: 0.0258 - acc: 0.9921 - val_loss: 0.0139 - val_acc: 0.9957
Q 858+795 T 1455 ☑ 1455
Q 019+348 T 1753 ☑ 1753
Q 317+929 T 1642 ☑ 1642
Q 703+631 T 443  ☑ 443 
Q 4+179   T 975  ☑ 975 
Q 152+306 T 854  ☑ 854 
Q 239+676 T 1608 ☑ 1608
Q 131+735 T 668  ☑ 668 
Q 761+665 T 733  ☑ 733 
Q 836+034 T 1068 ☑ 1068
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 19s 419us/step - loss: 0.0171 - acc: 0.9949 - val_loss: 0.0557 - val_acc: 0.9817
Q 957+439 T 1693 ☑

45000/45000 [==============================] - 19s 425us/step - loss: 0.0130 - acc: 0.9962 - val_loss: 0.0058 - val_acc: 0.9983
Q 916+538 T 1454 ☑ 1454
Q 059+44  T 994  ☑ 994 
Q 357+46  T 817  ☑ 817 
Q 346+32  T 666  ☑ 666 
Q 341+452 T 397  ☑ 397 
Q 8+51    T 23   ☑ 23  
Q 381+185 T 764  ☑ 764 
Q 001+469 T 1064 ☑ 1064
Q 479+389 T 1957 ☑ 1957
Q 852+541 T 403  ☑ 403 
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 19s 412us/step - loss: 0.0141 - acc: 0.9957 - val_loss: 0.0450 - val_acc: 0.9844
Q 205+7   T 509  ☑ 509 
Q 437+899 T 1732 ☑ 1732
Q 67+374  T 549  ☒ 449 
Q 554+856 T 1113 ☑ 1113
Q 383+214 T 795  ☑ 795 
Q 414+548 T 1259 ☑ 1259
Q 685+585 T 1171 ☑ 1171
Q 283+689 T 1368 ☑ 1368
Q 067+664 T 1226 ☑ 1226
Q 116+24  T 653  ☒ 643 
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 19s 414us/step - loss: 0.0142 - acc: 0.9960 - val_loss: 0.0043 - val_acc: 0.9984
Q 544+082 T 725  ☑

45000/45000 [==============================] - 19s 422us/step - loss: 0.0122 - acc: 0.9964 - val_loss: 0.0192 - val_acc: 0.9944
Q 995+268 T 1461 ☑ 1461
Q 546+304 T 1048 ☑ 1048
Q 527+194 T 1216 ☑ 1216
Q 26+017  T 772  ☑ 772 
Q 25+279  T 1024 ☑ 1024
Q 674+014 T 886  ☑ 886 
Q 308+507 T 1508 ☑ 1508
Q 562+972 T 544  ☑ 544 
Q 764+336 T 1100 ☑ 1100
Q 448+966 T 1513 ☑ 1513
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 19s 413us/step - loss: 0.0139 - acc: 0.9963 - val_loss: 0.0236 - val_acc: 0.9935
Q 933+998 T 1238 ☑ 1238
Q 76+433  T 401  ☑ 401 
Q 845+182 T 829  ☑ 829 
Q 826+831 T 766  ☑ 766 
Q 995+376 T 1272 ☑ 1272
Q 127+275 T 1293 ☑ 1293
Q 479+389 T 1957 ☑ 1957
Q 513+381 T 498  ☑ 498 
Q 354+113 T 764  ☑ 764 
Q 361+669 T 1129 ☑ 1129
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 19s 413us/step - loss: 0.0083 - acc: 0.9975 - val_loss: 0.0118 - val_acc: 0.9968
Q 685+774 T 1063 ☑

45000/45000 [==============================] - 19s 413us/step - loss: 0.0077 - acc: 0.9979 - val_loss: 0.0202 - val_acc: 0.9940
Q 062+823 T 588  ☑ 588 
Q 599+203 T 1297 ☑ 1297
Q 799+928 T 1826 ☑ 1826
Q 709+007 T 1607 ☑ 1607
Q 129+896 T 1619 ☑ 1619
Q 853+525 T 883  ☑ 883 
Q 702+289 T 1189 ☑ 1189
Q 388+603 T 1189 ☑ 1189
Q 907+838 T 1547 ☑ 1547
Q 832+527 T 963  ☑ 963 
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 19s 421us/step - loss: 0.0095 - acc: 0.9975 - val_loss: 0.0117 - val_acc: 0.9967
Q 077+862 T 1038 ☑ 1038
Q 955+749 T 1506 ☑ 1506
Q 16+76   T 128  ☑ 128 
Q 619+932 T 1155 ☑ 1155
Q 577+689 T 1761 ☑ 1761
Q 702+047 T 947  ☑ 947 
Q 487+103 T 1085 ☑ 1085
Q 605+031 T 636  ☑ 636 
Q 422+642 T 470  ☑ 470 
Q 214+15  T 463  ☑ 463 
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 19s 414us/step - loss: 0.0077 - acc: 0.9977 - val_loss: 0.0060 - val_acc: 0.9982
Q 069+7   T 967  ☑

45000/45000 [==============================] - 19s 414us/step - loss: 0.0090 - acc: 0.9975 - val_loss: 0.0044 - val_acc: 0.9988
Q 395+944 T 1042 ☑ 1042
Q 85+795  T 655  ☑ 655 
Q 941+277 T 921  ☑ 921 
Q 622+176 T 897  ☑ 897 
Q 413+183 T 695  ☑ 695 
Q 966+949 T 1618 ☑ 1618
Q 34+106  T 644  ☑ 644 
Q 945+024 T 969  ☑ 969 
Q 545+587 T 1330 ☑ 1330
Q 225+601 T 628  ☑ 628 
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 19s 419us/step - loss: 0.0053 - acc: 0.9986 - val_loss: 0.0058 - val_acc: 0.9985
Q 384+412 T 697  ☑ 697 
Q 422+856 T 882  ☑ 882 
Q 681+237 T 918  ☑ 918 
Q 151+162 T 412  ☑ 412 
Q 36+591  T 258  ☑ 258 
Q 073+848 T 1218 ☑ 1218
Q 558+57  T 930  ☑ 930 
Q 591+568 T 1060 ☑ 1060
Q 923+098 T 1219 ☑ 1219
Q 302+487 T 987  ☑ 987 
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 19s 416us/step - loss: 0.0098 - acc: 0.9975 - val_loss: 0.0049 - val_acc: 0.9985
Q 851+217 T 870  ☑

KeyboardInterrupt: 

下圖為以上架構的詳解, 圖源來自

https://qiita.com/HotAllure/items/0045998971a48909853d

<img style="float: left;" src="pics/encoder.png" width="50%">
<img style="float: right;" src="pics/decoder.png" width="50%">